# Translation (PyTorch)


*   https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt



Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs
!pip install sentencepiece
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

You will need to setup git, adapt your email and name in the following cell.

In [2]:
'''!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"'''

'!git config --global user.email "you@example.com"\n!git config --global user.name "Your Name"'

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [3]:
'''from huggingface_hub import notebook_login

notebook_login()'''

'from huggingface_hub import notebook_login\n\nnotebook_login()'

In [4]:
'''from datasets import load_dataset

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")'''

'from datasets import load_dataset\n\nraw_datasets = load_dataset("kde4", lang1="en", lang2="fr")'

In [5]:
from huggingface_hub.repocard import yaml
from datasets import load_dataset, Dataset, load_metric
import pandas as pd
from transformers import pipeline
import csv
import torch
import pandas as pd
import json
import numpy as np

def dataframe_to_jsonl(dataframe, jsonl_file_path):
    with open(jsonl_file_path, 'w') as jsonl_file:
        for _, row in dataframe.iterrows():
            json_object = row.to_json(orient='columns')
            jsonl_file.write(json_object + '\n')

colnames = ['sentence', 'label', 'data', 'regex', 'subset', 'lang_es']
train_df = pd.read_csv("COVID_Train_Set_Trans.csv", header=0, index_col=None, names=colnames, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, skiprows=1, engine='python', na_values='None')
train_df
print(train_df)
print(train_df.iloc[:5])
#train_df = train_df.loc[train_df["sentence"] != 'None']
train_df = train_df.dropna()
train_df['regex'] = train_df['regex'].map({True: 'True', False: 'False'})
train_df['data'] = train_df['data'].astype(object)
#train_df['translation']
#train_df = train_df.rename(columns={'sentence': "en", 'lang_es': 'es'})
test_df = pd.read_csv("COVID_Test_Set_Trans.csv", header=0, index_col=None, names=colnames, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, skiprows=1, engine='python', na_values='None')
print(test_df)
print(test_df.iloc[:5])
test_df = test_df.dropna()
#test_df = test_df.loc[test_df["sentence"] != 'None']
test_df['regex'] = test_df['regex'].map({True: 'True', False: 'False'})
test_df['data'] = test_df['data'].astype(object)
#test_df['translation']
#train_df = train_df.rename(columns={'sentence': "en", 'lang_es': 'es'})

# Example usage
  # Replace 'data.csv' with your DataFrame source
jsonl_file_path = 'train_lang.jsonl'
dataframe_to_jsonl(train_df, jsonl_file_path)

jsonl_file_path = 'test_lang.jsonl'
dataframe_to_jsonl(test_df, jsonl_file_path)

data_load = {"train": "train_lang.jsonl", "test": "test_lang.jsonl"}
# TypeError: expected str, bytes or os.PathLike object, not Dataset
covid_dataset = load_dataset("json", data_files=data_load)


<ipython-input-5-631f2ca2d001>:18: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train_df = pd.read_csv("COVID_Train_Set_Trans.csv", header=0, index_col=None, names=colnames, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, skiprows=1, engine='python', na_values='None')
Skipping line 148: Expected 7 fields in line 148, saw 8
Skipping line 162: Expected 7 fields in line 162, saw 8
Skipping line 164: Expected 7 fields in line 164, saw 8
Skipping line 165: Expected 7 fields in line 165, saw 8
Skipping line 217: Expected 7 fields in line 217, saw 9
Skipping line 221: Expected 7 fields in line 221, saw 8
Skipping line 251: Expected 7 fields in line 251, saw 8
Skipping line 319: Expected 7 fields in line 319, saw 8
Skipping line 397: Expected 7 fields in line 397, saw 8
Skipping line 484: Expected 7 fields in line 484, saw 8
Skipping line 496: Expected 7 fields in lin

                                                sentence  label        data  \
7                 SARS-CoV-2 Omicron Variant of Concern      1  SARS-CoV-2   
65                SARS-CoV-2 Omicron Variant of Concern      1  SARS-CoV-2   
70     Table 2. Monoclonal Antibodies for Treatment o...      1    COVID-19   
76     02/11/2022 Mild-to-moderate COVID-19 in adults...      1    COVID-19   
78     positive results who are at high risk for prog...      1    positive   
...                                                  ...    ...         ...   
29503  KEYWORDS Science of trust; trust; health commu...      1      social   
29511  I will also echo what I think Gil initially sa...      1   community   
29536  Existing Research Gaps/Areas for Further Explo...      1        Area   
29560  [5] Schiavo R. 2021. Challenging misinformatio...      1   community   
29562  [17] Shapin S. A social history of truth. Chic...      1      social   

       regex subset                                

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b53fae84de849759/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
'''def dataframe_to_jsonl(dataframe, jsonl_file_path):
    with open(jsonl_file_path, 'w') as jsonl_file:
        for _, row in dataframe.iterrows():
            json_object = row.to_json(orient='columns')
            jsonl_file.write(json_object + '\n')

# Example usage
  # Replace 'data.csv' with your DataFrame source
jsonl_file_path = 'train.jsonl'
dataframe_to_jsonl(train_df, jsonl_file_path)

jsonl_file_path = 'test.jsonl'
dataframe_to_jsonl(test_df, jsonl_file_path)

data_load = {"train": "train.jsonl", "test": "test.jsonl"}
# TypeError: expected str, bytes or os.PathLike object, not Dataset
covid_dataset = load_dataset("json", data_files=data_load)'''

'def dataframe_to_jsonl(dataframe, jsonl_file_path):\n    with open(jsonl_file_path, \'w\') as jsonl_file:\n        for _, row in dataframe.iterrows():\n            json_object = row.to_json(orient=\'columns\')\n            jsonl_file.write(json_object + \'\n\')\n\n# Example usage\n  # Replace \'data.csv\' with your DataFrame source\njsonl_file_path = \'train.jsonl\'\ndataframe_to_jsonl(train_df, jsonl_file_path)\n\njsonl_file_path = \'test.jsonl\'\ndataframe_to_jsonl(test_df, jsonl_file_path)\n\ndata_load = {"train": "train.jsonl", "test": "test.jsonl"}\n# TypeError: expected str, bytes or os.PathLike object, not Dataset\ncovid_dataset = load_dataset("json", data_files=data_load)'

In [7]:
'''import pandas as pd
#df = pd.read_csv("21-1461.csv", encoding="utf-8", index_col=0)
#print(df)
#x = df.to_string(header=True, index=True)
#print(x)
train_total = train_df.shape[0] - 1
train_pal = train_df['sentence'].values.tolist()
print(train_pal)
test_total = test_df.shape[0] - 1
test_pal = test_df['sentence'].values.tolist()
print(test_pal)

def classify_text(data):
    text = data["sentence"]
    result = classifier(text, labels)
    return result["labels"][0]
'''
'''
#data2 = {'text': [pal]}

#new_list = [item.split() for item in pal]
#print(new_list)
#lst = pal
#s = "".join(lst)
#result = s.split('!')
#print(result)
#myList = [i.split('\t')[0] for i in pal]
#print(pal)
#for pal in df['sentence']:
#df['lang_ru'] = df['sentence'].apply(lambda pal :translator(pal, max_length=65990))
#data = data.applymap(lambda pal: translator(pal)[0]['translation_text'])
train_df['lang_es'] = train_df['sentence'].apply(lambda train_pal :translator(train_pal)[0]['translation_text'])
test_df['lang_es'] = test_df['sentence'].apply(lambda test_pal :translator(test_pal)[0]['translation_text'])
#data["sentiment"] = data["sentence"].apply(lambda pal: classifier(pal)[0]['label'])
#data["zero-shot"] = data["sentence"].apply(lambda pal: zsclassifier(pal, labels)["scores"])
#data["generated"] = data["sentence"].apply(lambda pal: generator(pal, max_length=50)[0]["generated_text"])
print(train_df)
print(test_df)
train_df.to_csv("COVID_Train_Set_Trans.csv")
test_df.to_csv("COVID_Test_Set_Trans.csv")
#print(df)

#translated = translator(vals).get('translation_text')
#print(translated)
#for x in df['sentence']:
# print(translator(i))
#df['lang-ru'] = [translator(i) for x]
#print(df)
#df.to_csv('Translated_column.csv')'''


'\n#data2 = {\'text\': [pal]}\n\n#new_list = [item.split() for item in pal]\n#print(new_list)\n#lst = pal\n#s = "".join(lst)\n#result = s.split(\'!\')\n#print(result)\n#myList = [i.split(\'\t\')[0] for i in pal] \n#print(pal)\n#for pal in df[\'sentence\']:\n#df[\'lang_ru\'] = df[\'sentence\'].apply(lambda pal :translator(pal, max_length=65990))\n#data = data.applymap(lambda pal: translator(pal)[0][\'translation_text\'])\ntrain_df[\'lang_es\'] = train_df[\'sentence\'].apply(lambda train_pal :translator(train_pal)[0][\'translation_text\'])\ntest_df[\'lang_es\'] = test_df[\'sentence\'].apply(lambda test_pal :translator(test_pal)[0][\'translation_text\'])\n#data["sentiment"] = data["sentence"].apply(lambda pal: classifier(pal)[0][\'label\'])\n#data["zero-shot"] = data["sentence"].apply(lambda pal: zsclassifier(pal, labels)["scores"])\n#data["generated"] = data["sentence"].apply(lambda pal: generator(pal, max_length=50)[0]["generated_text"])\nprint(train_df)\nprint(test_df)\ntrain_df.to_csv

In [8]:
covid_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'lang_es'],
        num_rows: 2608
    })
    test: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'lang_es'],
        num_rows: 736
    })
})

In [9]:
split_datasets = covid_dataset["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'lang_es'],
        num_rows: 2347
    })
    test: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'lang_es'],
        num_rows: 261
    })
})

In [10]:
split_datasets["test"] = split_datasets.pop("test")

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig, AutoModel, Pipeline
'''
 # Wrong code
src = "en"
dst = "es"

task_name = f"translation_{src}_to_{dst}"
model_name = f"Helsinki-NLP/opus-mt-{src}-{dst}"
#bart_config = BartConfig(
#    vocab_size=vocab_size, d_model=64, encoder_ffn_dim=64, encoder_layers=2, decoder_ffn_dim=64, decoder_layers=2
#)
#config = bart_config.to_dict()
#model = AutoModelForSeq2SeqLM.from_config(BartConfig.from_dict(config))
translator  = pipeline(task_name, model=model_name, tokenizer=model_name)
#vocab_size = 99000
#num_new_tokens = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")
#model.resize_token_embeddings(model.config.vocab_size + num_new_tokens)
config = AutoConfig.from_pretrained("Helsinki-NLP/opus-mt-en-es")

print(model.config)
model.config.vocab_size
#print(vocab_size)'''

'\n # Wrong code\nsrc = "en"\ndst = "es"\n\ntask_name = f"translation_{src}_to_{dst}"\nmodel_name = f"Helsinki-NLP/opus-mt-{src}-{dst}"\n#bart_config = BartConfig(\n#    vocab_size=vocab_size, d_model=64, encoder_ffn_dim=64, encoder_layers=2, decoder_ffn_dim=64, decoder_layers=2\n#)\n#config = bart_config.to_dict()\n#model = AutoModelForSeq2SeqLM.from_config(BartConfig.from_dict(config))\ntranslator  = pipeline(task_name, model=model_name, tokenizer=model_name)\n#vocab_size = 99000\n#num_new_tokens = 10\ndevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\ntokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")\n\nmodel = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")\n#model.resize_token_embeddings(model.config.vocab_size + num_new_tokens)\nconfig = AutoConfig.from_pretrained("Helsinki-NLP/opus-mt-en-es")\n\nprint(model.config)\nmodel.config.vocab_size\n#print(vocab_size)'

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig, pipeline

src = "en"
dst = "es"

task_name = f"translation_{src}_to_{dst}"
model_name = f"Helsinki-NLP/opus-mt-{src}-{dst}"

max_length = 512
model_checkpoint = "Helsinki-NLP/opus-mt-en-es"
#tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt", max_length=512)
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")
config = AutoConfig.from_pretrained("Helsinki-NLP/opus-mt-en-es")

translator = pipeline(task_name, model=model, tokenizer=tokenizer, config=config)
'''
en_sentence = split_datasets["train"]["sentence"]
print(en_sentence)
es_sentence = split_datasets["train"]["lang_es"]
print(es_sentence)
'''

en_sentence = split_datasets["train"]["sentence"][1]
print(en_sentence)
es_sentence = split_datasets["train"]["lang_es"][1]
print(es_sentence)

#inputs = tokenizer(en_sentence, text_target=es_sentence, max_length=max_length, truncation=True, padding="max_length")
inputs = tokenizer(en_sentence, text_target=es_sentence)
print(inputs)

split_datasets["train"][170]["sentence"]

translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)
'''
wrong_targets = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
print(wrong_targets)
try:
  wrong_targets = tokenizer(es_sentence)
except Exception as e:
  print(wrong_targets)
  raise(e)
'''
wrong_targets = tokenizer(es_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))


Brief internetdelivered cognitivebehavioural intervention for children and adolescents with symptoms of anxiety and depression during the COVID19 pandemic: a randomised controlled trial protocol
Breve intervención cognoscitiva de comportamiento en internet para niños y adolescentes con síntomas de ansiedad y depresión durante la pandemia COVID19: un protocolo de ensayo controlado aleatorizado
{'input_ids': [43616, 4448, 8092, 118, 32233, 1512, 2139, 10304, 19376, 6734, 23, 348, 10, 10934, 41, 7079, 7, 21235, 10, 17754, 478, 5, 202, 13285, 5949, 2681, 28885, 34, 8, 53772, 7484, 4399, 10936, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [5578, 243, 4447, 836, 10735, 9, 1004, 3184, 4, 6100, 12, 4448, 24, 396, 11, 6367, 29, 6797, 4, 21166, 11, 17222, 314, 6, 29798, 4322, 6503, 615, 2681, 34, 28, 10490, 4, 7523, 20608, 49094, 0]}
['▁Bre', 've', '▁inter', 'ven', 'ci', 'ó', 'n', '▁co', 'gno', 's', 'ci', 

In [13]:
'''max_length = 128


def preprocess_function(examples):
    inputs = [ex["sentence"] for ex in examples["sentence"]]
    targets = [ex["lang_es"] for ex in examples["lang_es"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs'''

'max_length = 128\n\n\ndef preprocess_function(examples):\n    inputs = [ex["sentence"] for ex in examples["sentence"]]\n    targets = [ex["lang_es"] for ex in examples["lang_es"]]\n    model_inputs = tokenizer(\n        inputs, text_target=targets, max_length=max_length, truncation=True\n    )\n    return model_inputs'

In [14]:
'''max_length = 128

def preprocess_function(examples):
    inputs = [train_df["sentence"] for ex in examples]
    targets = [train_df["lang_es"] for ex in examples]

    model_inputs = tokenizer(
        inputs,
        text_target=targets,
        max_length=max_length,
        truncation=True,
        padding="max_length",
    )

    return model_inputs'''

'max_length = 128\n\ndef preprocess_function(examples):\n    inputs = [train_df["sentence"] for ex in examples]\n    targets = [train_df["lang_es"] for ex in examples]\n    \n    model_inputs = tokenizer(\n        inputs,\n        text_target=targets,\n        max_length=max_length,\n        truncation=True,\n        padding="max_length",\n    )\n    \n    return model_inputs'

In [15]:
'''tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)'''

'tokenized_datasets = split_datasets.map(\n    preprocess_function,\n    batched=True,\n    remove_columns=split_datasets["train"].column_names,\n)'

In [16]:
'''max_length = 128

def preprocess_function(examples):
    inputs = [examples["sentence"]]
    targets = [examples["lang_es"]]
    print(inputs[0])
    print(targets[0])
    model_inputs = tokenizer(
        inputs,
        text_target=targets,
        max_length=max_length,
        truncation=True,
        padding="max_length",
    )

    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)'''


'max_length = 128\n\ndef preprocess_function(examples):\n    inputs = [examples["sentence"]]\n    targets = [examples["lang_es"]]\n    print(inputs[0])\n    print(targets[0])\n    model_inputs = tokenizer(\n        inputs,\n        text_target=targets,\n        max_length=max_length,\n        truncation=True,\n        padding="max_length",\n    )\n    \n    return model_inputs\n\ntokenized_datasets = split_datasets.map(\n    preprocess_function,\n    batched=True,\n    remove_columns=split_datasets["train"].column_names,\n)'

In [17]:
max_length = 128

def preprocess_function(examples):
  inputs = [sentence for sentence in examples['sentence']]
  targets = [sentence for sentence in examples['lang_es']]
  model_inputs = tokenizer(
      inputs,
      text_target=targets,
      max_length=max_length,
      truncation=True,
      padding="max_length",
      )
  return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)


Map:   0%|          | 0/2347 [00:00<?, ? examples/s]

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

In [18]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [19]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [21]:
batch["labels"]

tensor([[ 5578,   243,  4447,   836, 10735,     9,  1004,  3184,     4,  6100,
            12,  4448,    24,   396,    11,  6367,    29,  6797,     4, 21166,
            11, 17222,   314,     6, 29798,  4322,  6503,   615,  2681,    34,
            28, 10490,     4,  7523, 20608, 49094,     0, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 6

In [22]:
batch["decoder_input_ids"]

tensor([[65000,  5578,   243,  4447,   836, 10735,     9,  1004,  3184,     4,
          6100,    12,  4448,    24,   396,    11,  6367,    29,  6797,     4,
         21166,    11, 17222,   314,     6, 29798,  4322,  6503,   615,  2681,
            34,    28, 10490,     4,  7523, 20608, 49094,     0, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 6

In [23]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[5578, 243, 4447, 836, 10735, 9, 1004, 3184, 4, 6100, 12, 4448, 24, 396, 11, 6367, 29, 6797, 4, 21166, 11, 17222, 314, 6, 29798, 4322, 6503, 615, 2681, 34, 28, 10490, 4, 7523, 20608, 49094, 0, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000]
[22015, 201, 69, 3782, 4, 12901, 836, 12695, 7750, 11, 2156, 1670, 29, 14, 274, 11, 6, 330, 4209, 836, 12695, 7750, 29, 20157, 477, 0, 65000, 65000, 65000, 65000, 65000, 

In [24]:
!pip install sacrebleu

from evaluate import load as load_metric

metric = load_metric("sacrebleu")

sentences1 = train_df.iloc[0, 0]
example1 = "".join(sentences1)

sentences2 = train_df.iloc[2, 0]
example2 = "".join(sentences2)

predictions = [example1]
references = [[example2]]

metric_result = metric.compute(predictions=predictions, references=references)

print(metric_result)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 6.4 MB/s eta 0:00:00


{'score': 3.2174093287959424, 'counts': [1, 0, 0, 0], 'totals': [5, 4, 3, 2], 'precisions': [20.0, 12.5, 8.333333333333334, 6.25], 'bp': 0.301194211912202, 'sys_len': 5, 'ref_len': 11}


In [25]:
sentences3 = train_df.iloc[3, 0]
example3 = "".join(sentences3)

sentences4 = train_df.iloc[4, 0]
example4 = "".join(sentences4)

predictions1 = [example3]
references1 = [[example4]]

metric_result1 = metric.compute(predictions=predictions1, references=references1)

print(metric_result1)

{'score': 0.0, 'counts': [0, 0, 0, 0], 'totals': [13, 12, 11, 10], 'precisions': [0.0, 0.0, 0.0, 0.0], 'bp': 1.0, 'sys_len': 13, 'ref_len': 11}


In [26]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [27]:
'''from huggingface_hub import notebook_login
'
notebook_login()'''

"from huggingface_hub import notebook_login\n'\nnotebook_login()"

In [28]:
from transformers import Seq2SeqTrainingArguments

'''args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-es",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)'''

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-es",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

In [29]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [30]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 5.3087687492370605,
 'eval_bleu': 96.82817115497394,
 'eval_runtime': 38.1519,
 'eval_samples_per_second': 6.841,
 'eval_steps_per_second': 0.131}

In [31]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=222, training_loss=0.23418612093538851, metrics={'train_runtime': 83.3162, 'train_samples_per_second': 84.509, 'train_steps_per_second': 2.665, 'total_flos': 238678387458048.0, 'train_loss': 0.23418612093538851, 'epoch': 3.0})

In [32]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 0.032816555351018906,
 'eval_bleu': 85.79557036436222,
 'eval_runtime': 24.6244,
 'eval_samples_per_second': 10.599,
 'eval_steps_per_second': 0.203,
 'epoch': 3.0}

In [33]:
'''trainer.push_to_hub(tags="translation", commit_message="Training complete")'''

'trainer.push_to_hub(tags="translation", commit_message="Training complete")'

In [34]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=8
)

In [35]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [36]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [37]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [38]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [39]:
'''from huggingface_hub import Repository, get_full_repo_name

model_name = "marian-finetuned-kde4-en-to-es-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name'''

'from huggingface_hub import Repository, get_full_repo_name\n\nmodel_name = "marian-finetuned-kde4-en-to-es-accelerate"\nrepo_name = get_full_repo_name(model_name)\nrepo_name'

In [40]:
'''output_dir = "marian-finetuned-kde4-en-to-es-accelerate"
repo = Repository(output_dir, clone_from=repo_name)'''

'output_dir = "marian-finetuned-kde4-en-to-es-accelerate"\nrepo = Repository(output_dir, clone_from=repo_name)'

In [41]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [43]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")
'''
    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )'''

  0%|          | 0/882 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

epoch 0, BLEU score: 84.90


  0%|          | 0/33 [00:00<?, ?it/s]

epoch 1, BLEU score: 84.58


  0%|          | 0/33 [00:00<?, ?it/s]

epoch 2, BLEU score: 84.58


'\n    # Save and upload\n    accelerator.wait_for_everyone()\n    unwrapped_model = accelerator.unwrap_model(model)\n    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)\n    if accelerator.is_main_process:\n        tokenizer.save_pretrained(output_dir)\n        repo.push_to_hub(\n            commit_message=f"Training in progress epoch {epoch}", blocking=False\n        )'

In [45]:
'''from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "huggingface-course/marian-finetuned-kde4-en-to-es"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")'''

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_errors.py:259 in                  │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   256 │   </Tip>                                                                                 │
│   257 │   """                                                                                    │
│   258 │   try:                                                                                   │
│ ❱ 259 │   │   response.raise_for_status()                                                        │
│   260 │   except HTTPError as e:                                                                 │
│   261 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   262                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/requests/models.py:960 in raise_for_status               │
│                                                                                                  │
│   957 │   │   │   http_error_msg = u'%s Server Error: %s for url: %s' % (self.status_code, rea   │
│   958 │   │                                                                                      │
│   959 │   │   if http_error_msg:                                                                 │
│ ❱ 960 │   │   │   raise HTTPError(http_error_msg, response=self)                                 │
│   961 │                                                                                          │
│   962 │   def close(self):                                                                       │
│   963 │   │   """Releases the connection back to the pool. Once this method has been             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 401 Client Error: Unauthorized for url: 
https://huggingface.co/huggingface-course/marian-finetuned-kde4-en-to-es/resolve/main/config.json

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:417 in cached_file             │
│                                                                                                  │
│    414 │   user_agent = http_user_agent(user_agent)                                              │
│    415 │   try:                                                                                  │
│    416 │   │   # Load from URL or cache if already cached                                        │
│ ❱  417 │   │   resolved_file = hf_hub_download(                                                  │
│    418 │   │   │   path_or_repo_id,                                                              │
│    419 │   │   │   filename,                                                                     │
│    420 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:118 in _inner_fn    │
│                                                                                                  │
│   115 │   │   if check_use_auth_token:                                                           │
│   116 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   117 │   │                             

In [ ]:
'''translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)'''